In [35]:
# Background: I love VW Beetles and I love data!  I have a 1958 in my garage and this project is more of a passion for me.
#             I found that pricing data for classic VW sales was either expensive, incomplete, or difficult to access.
#             So, I decided that I would curate a pricing dataset myself for Beetles (Type-1's) of all years. 
#             It is updated several times each week and originates from hundreds of auction houses around the world. 
#             The data may not be exhaustive, but it's a good sample!
#
# Objective:  To create a dataset that I can use to generate analytical insights for the benefit of VW Beetle enthusiasts. Enjoy! 

# Let's import some libraries that we'll us throughout the workflow.
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urlparse, parse_qs
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import re
from datetime import datetime

# Function to cleanup and normalize the Mileage field.
def process_mileage(mileage_text):
    if 'TMU' in mileage_text:
        return ''
    mileage_text = mileage_text.replace(' mi', '')
    if 'k mi' in mileage_text:
        return mileage_text.replace('k mi', ',000')
    if 'k km' in mileage_text:
        match = re.search(r'\((\d+k)\)', mileage_text)
        if match:
            return match.group(1).replace('k', ',000')
        match = re.search(r'\((\d+)\)', mileage_text)
        if match:
            return f"{int(match.group(1)):,}"
    if 'k' in mileage_text:
        return mileage_text.replace('k', ',000')
    if mileage_text == 'N/A':
        return ''
    return mileage_text

# Function to extract the year from the title
def extract_year(title):
    match = re.search(r'\b\d{4}\b', title)
    if match and match.group().isdigit():
        return match.group()
    return 'N/A'

# Function to convert date to proper date format
def process_date(date_text):
    try:
        return datetime.strptime(date_text, '%b %d, %Y').date()
    except ValueError:
        return None

# Function to extract data from a single page using HTML elements on the page to isolate the elements I wanted to scrape.
def extract_data_from_page(soup):
    data = []
    listings_section = soup.find('div', class_='listings-table')
    if not listings_section:
        return data
    
    listings = listings_section.find_all('div', class_='border border-gray-300 rounded-lg shadow-lg h-full flex flex-col justify-between table:h-auto table:flex-row table:rounded-none table:border-none table:shadow-none table:px-4')
    
    for listing in listings:
        title_tag = listing.find('a', class_='text-xl leading-5 font-medium table:text-secondary table:text-base flex-1')
        
        # Extract the price
        price_tag_black = listing.find('div', class_='text-sm font-medium text-black')
        price_tag_red = listing.find('div', class_='text-sm font-medium text-red-600')
        
        price_tag = price_tag_black if price_tag_black else price_tag_red
        
        # Extract the mileage
        mileage_divs = listing.find_all('div', class_='flex items-center')
        mileage_tag = next((div.get_text(strip=True) for div in mileage_divs if 'mi' in div.get_text(strip=True) or 'km' in div.get_text(strip=True)), 'N/A')
        mileage_tag = process_mileage(mileage_tag)
        
        # Extract the location
        location_div = listing.find('div', class_='hidden table:block')
        location_tag = location_div.find('div', class_='flex gap-2 items-center text-gray-500').find('div').text.strip() if location_div else 'N/A'
        
        # Extract the condition
        condition_div = listing.find('div', class_='uppercase font-semibold text-xs')
        condition_tag = condition_div.find('abbr', class_='no-underline').text.strip() if condition_div else 'N/A'
        
        # Extract the status
        status_classes = [
            'border font-medium uppercase inline-block whitespace-nowrap text-black bg-gray-200 px-1 py-0.5 text-sm rounded',
            'border font-medium uppercase inline-block whitespace-nowrap text-green-600 border-green-600 px-1 py-0.5 text-sm rounded',
            'border font-medium uppercase inline-block whitespace-nowrap text-red-600 border-red-600 px-1 py-0.5 text-sm rounded',
            'border font-medium uppercase inline-block whitespace-nowrap text-white bg-black border-black px-1 py-0.5 text-sm rounded',
            'border font-medium uppercase inline-block whitespace-nowrap text-black bg-gray-200 px-1 py-0.5 text-sm rounded'
        ]
        status_tag = None
        for status_class in status_classes:
            status_tag = listing.find('div', class_=status_class)
            if status_tag:
                break
        status_text = status_tag.text.strip() if status_tag else 'N/A'
        
        # Extract the Auctioneer
        auctioneer_tag = listing.find('a', class_='hover:underline table:text-black table:font-medium')
        
        # Extract the Date
        date_tag = listing.find('span', class_='table:text-black')
        date_text = process_date(date_tag.text.strip()) if date_tag else None
        
        # Extract the Link
        link = title_tag['href'] if title_tag else None
        
        # Extract the US dollar price
        price_text = price_tag.text.strip() if price_tag else 'N/A'
        us_dollar_price = re.search(r'\$(\d{1,3}(?:,\d{3})*(?:\.\d{2})?)', price_text)
        us_dollar_price = us_dollar_price.group(0) if us_dollar_price else ''
        
        # Extract the year from the title
        year = extract_year(title_tag.text.strip()) if title_tag else 'N/A'
        
        data.append({
            'Title': title_tag.text.strip() if title_tag else 'N/A',
            'Year': year,
            'Price': us_dollar_price,
            'Location': location_tag,
            'Condition': condition_tag,
            'Mileage': mileage_tag,
            'Status': status_text,
            'Auctioneer': auctioneer_tag.text.strip() if auctioneer_tag else 'N/A',
            'Date': date_text,
            'Link': link
        })
    return data

# Function to handle pagination
def scrape_all_pages(base_url):
    all_data = []
    page = 1
    visited_pages = set()
    
    while True:
        url = f"{base_url}?page={page}"
        response = requests.get(url)
        if response.status_code != 200:
            break
        soup = BeautifulSoup(response.content, 'html.parser')
        data = extract_data_from_page(soup)
        if not data:
            break
        all_data.extend(data)

        next_button = soup.find('a', rel='next')
        if not next_button:
            break
        
        next_page_url = next_button['href']
        
        parsed_url = urlparse(next_page_url)
        next_page_number = int(parse_qs(parsed_url.query).get('page', [None])[0])
        
        if next_page_number in visited_pages:
            break

        visited_pages.add(next_page_number)
        page = next_page_number
        
    return all_data

# Function to authenticate and write data to Google Sheets
def write_to_google_sheets(data):
    try:
        # Authenticate using the vw-beetle-scraper-1a0bb7d8e2f9.json file
        scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
        creds = ServiceAccountCredentials.from_json_keyfile_name('vw-beetle-scraper-1a0bb7d8e2f9.json', scope)
        client = gspread.authorize(creds)
        
        # Open the Google Sheet (use your sheet name or ID)
        sheet = client.open("VW Beetles Auction Listings").sheet1
        
        # Clear the sheet before writing new data
        sheet.clear()
        
        # Prepare the data
        headers = data[0].keys()
        rows = [list(row.values()) for row in data]
        
        # Write the headers and rows in a single batch
        sheet.append_row(list(headers))
        sheet.append_rows(rows)
        
        print("Data has been written to the Google Sheet")
    except gspread.exceptions.APIError as api_err:
        print(f"An API error occurred: {api_err}")
        print(f"API Error details: {api_err.response.text}")
    except gspread.exceptions.GSpreadException as gs_err:
        print(f"A GSpread exception occurred: {gs_err}")
    except Exception as e:
        print(f"An unexpected error occurred while writing to Google Sheets: {e}")

# Function to write data to CSV
def write_to_csv(data):
    try:
        df = pd.DataFrame(data)
        df.to_csv('vw_beetles_auction_listings.csv', index=False)
        print("Data has been written to vw_beetles_auction_listings.csv")
    except Exception as e:
        print(f"An error occurred while writing to CSV: {e}")

# Main function to run the scraper and write data to Google Sheets and CSV
def main():
    base_url = 'https://www.classic.com/m/volkswagen/beetle/type-1/'
    try:
        all_data = scrape_all_pages(base_url)
        if all_data:
            write_to_google_sheets(all_data)
            write_to_csv(all_data)
        else:
            print('No data extracted')
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == '__main__':
    main()
    print("Script has finished.")


Data has been written to the Google Sheet
Data has been written to vw_beetles_auction_listings.csv
Script has finished.
